# Preventing Attribute Creation
If we want to prevent objects from being able to create new attributes after they are initialized, we can alter the objects' `__setattr__()` method. All attribute assignment statements for an instance go through the `__setattr__()` method of the instance. Since attribute creation is also implicitly done via attribute assignment, this is the place to add the logic we desire.

Preventing attribute creation after an object is initialized corresponds to the more "locked down" style of languages like Java. Doing so has the advantage of preventing client code from mistakenly creating a new attribute when the real intent was to assign to an existing attribute (e.g. mistakenly creating a `length` attribute when the real intent was to assign to an existing `height` attribute). However, this also closes off some flexibility, and there are valid use cases that take advantage of this flexibility of open attribute creation (see the "advantages_of_open_attribute_creation" tutorial).

Below is an example that shows how to restrict attribute creation and assignment to a pre-defined set of attributes.

In [1]:
class Rectangle(object):
    def __setattr__(self, key, value):
        """
        Restrict attribute assignments to just "width" and "height"
        """
        assert key in ('width', 'height'), "Invalid attribute: " + key
        super().__setattr__(key, value)
        
    
    def __init__(self, width, height):
        self.width = width
        self.height = height

# Example
# Create a Rectangle
rectangle = Rectangle(1.2, 3.4) 
print(rectangle.width)

# We can change its height
rectangle.height = 5.6          
print(rectangle.height)  

# But we can't add a new attribute
rectangle.length = 10
print(rectangle.length)


1.2
5.6


AssertionError: Invalid attribute: length

In the code above, `__setattr__()` intercepts all attempts to set attributes on the Rectangle class and only actually sets attributes that it knows about: width and height, raising an error for all other attempts. 

Some people use the `__slots__` class variable to achieve the above behavior; however, that is not the intended purpose of `__slots__`, which has other limitations

# A do-nothing `__setattrr__`
As an aside, if we defined a 'do nothing" `__setattr__()` method, then no one would be able to set any attributes on instances of the class, not even the class itself, nor would any errors be raised when an assignment occurred.

In [2]:
class Rectangle(object):
    def __setattr__(self, key, value):
        """
        Prevent all attribute assignments, silently."
        """
        pass
        
    
    def __init__(self, width, height):
        self.width = width   # This "assignment" raises no error, but doesn't actually assign anything!
        self.height = height # This "assignment" raises no error, but doesn't actually assign anything!

# Example
# Create a Rectangle
rectangle = Rectangle(1.2, 3.4) 
print(rectangle.width) # This causes an error, because the attribute doesn't exist.

AttributeError: 'Rectangle' object has no attribute 'width'

# Freezing A Class
Rather than hard-coding the allowed member variables into the `__setattr__()` method, which creates coupling between the init method and the setattr method (every time we add or delete a member variable in the init method, we also have to change the setattr method), we can "freeze" our instance's set of member variables to just those that exist after the init method has done its job, as illustrated in the code below.

In [3]:
class Rectangle(object):
    def __setattr__(self, key, value):
        """
        Restrict attribute assignments to just those that exist at the time
        we are frozen.
        """
        if hasattr(self, '_frozen'):
            assert hasattr(self, key), "Invalid attribute: " + key
            
        super().__setattr__(key, value)
        
    
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self._frozen = True

# Example
# Create a Rectangle
rectangle = Rectangle(1.2, 3.4) 
print(rectangle.width)

# We can change its height
rectangle.height = 5.6          
print(rectangle.height)  

# But we can't add a new attribute
rectangle.length = 10
print(rectangle.length)


1.2
5.6


AssertionError: Invalid attribute: length

A more elegant version of the above, which hides implementation details, and doesn't have to be re-implemented for each class, is below. It wraps up the freeze functionality as a mixin.

In [4]:
class FrozenClassMixin(object):
    def _is_frozen_for_new_attributes(self):
        """
        Return True if we are frozen from creating new attributes, else False.
        """
        return hasattr(self, '_frozen_for_new_attributes')

    def freeze_new_attributes(self):
        """
        Set a flag on ourselves indicating creation of new attributes is no longer allowed
        """
        setattr(self, '_frozen_for_new_attributes', True)

    def __setattr__(self, key, value):
        """
        Disallow creation of new attributes on ourselves after freeze_new_attributes() has
        been called.
        """
        if self._is_frozen_for_new_attributes():
            assert hasattr(self, key), "Invalid attribute: " + key

        super().__setattr__(key, value)    

        
class Rectangle(FrozenClassMixin):    
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.freeze_new_attributes()


# Example
# Create a Rectangle
rectangle = Rectangle(1.2, 3.4) 
print(rectangle.width)

# We can change its height
rectangle.height = 5.6          
print(rectangle.height)  

# But we can't add a new attribute
rectangle.length = 10
print(rectangle.length)


1.2
5.6


AssertionError: Invalid attribute: length